# Database Snapshots

dbatools makes using database snapshots easier than ever.

- Take a snapshot of database
- Drop a table or run a rogue update
- Get the table back

## View our database snapshots
Database snapshots have a `source_database_id` value that relates to the database they were created from.

In [1]:
select name, database_id, source_database_id, is_read_only
from sys.databases
where name like 'AdventureWorks2017%'

(2 rows affected)

Total execution time: 00:00:00.053

name,database_id,source_database_id,is_read_only
AdventureWorks2017,5,NULL,0
AdventureWorks2017_20211011_230632,7,5,1


## View the current state

In [2]:
USE AdventureWorks2017

SELECT TOP 10 *
FROM Person.PersonPhone

SELECT TOP 10 *
FROM Person.Password

(10 rows affected)

(10 rows affected)

Total execution time: 00:00:00.050

BusinessEntityID,PasswordHash,PasswordSalt,rowguid,ModifiedDate
1,pbFwXWE99vobT6g+vPWFy93NtUU/orrIWafF01hccfM=,bE3XiWw=,329eacbe-c883-4f48-b8b6-17aa4627efff,2009-01-07 00:00:00.000
2,bawRVNrZQYQ05qF05Gz6VLilnviZmrqBReTTAGAudm0=,EjJaC3U=,a4c82398-7466-4fe6-b9ee-cec34d116f68,2008-01-24 00:00:00.000
3,8BUXrZfDqO1IyHCWOYzYmqN1IhTUn3CJMpdx/UCQ3iY=,wbPZqMw=,ac3f4536-bb2e-41c5-b70d-454be460c1bd,2007-11-04 00:00:00.000
4,SjLXpiarHSlz+6AG+H+4QpB/IPRzras/+9q/5Wr7tf8=,PwSunQU=,b3fa4c24-2e96-477c-a923-0cb0f6fa5c80,2007-11-28 00:00:00.000
5,8FYdAiY6gWuBsgjCFdg0UibtsqOcWHf9TyaHIP7+paA=,qYhZRiM=,c4d13bcf-0209-44c7-ac67-6f817fdd7f16,2007-12-30 00:00:00.000
6,u5kbN5n84NRE1h/a+ktdRrXucjgrmfF6wZC4g82rjHM=,a9GiLUA=,bbe788b9-8d6d-4799-87a7-7b85b6bd67dc,2013-12-16 00:00:00.000
7,zSqerln8T8eq3nYHC4Lx4vMuxZaxkDylVwWnP2ZT6QA=,13mu8BA=,afd3a20a-787b-4069-92db-aee666c02847,2009-02-01 00:00:00.000
8,s+FUWADIZzXBKpcbxe4OwL2uiJmjLogJNYXXHvc1X/k=,FlCpzTU=,4fe51b60-130e-4209-9e67-28dc4d91446c,2008-12-22 00:00:00.000
9,fCvCTy3RwzA2LNhhhYUbT7erkb9Au5wyM2q7ReHroV0=,FTcZMvQ=,99f4f320-f05d-4fa1-bb0d-81c9425422f3,2009-01-09 00:00:00.000
10,/8biMrxuAtETGeIuloSrMQHBraZtZ+eU2z5OJ1Fhn6M=,K7dMpTY=,82f25f0c-5d75-4246-958a-b6df67559d09,2009-04-26 00:00:00.000


BusinessEntityID,PhoneNumber,PhoneNumberTypeID,ModifiedDate
1,697-555-0142,1,2009-01-07 00:00:00.000
2,819-555-0175,3,2008-01-24 00:00:00.000
3,212-555-0187,1,2007-11-04 00:00:00.000
4,612-555-0100,1,2007-11-28 00:00:00.000
5,849-555-0139,1,2007-12-30 00:00:00.000
6,122-555-0189,3,2013-12-16 00:00:00.000
7,181-555-0156,3,2009-02-01 00:00:00.000
8,815-555-0138,1,2008-12-22 00:00:00.000
9,185-555-0186,1,2009-01-09 00:00:00.000
10,330-555-2568,3,2009-04-26 00:00:00.000


## Time for some fun - Rogue changes

In [3]:
Use AdventureWorks2017

DROP TABLE person.PASSWORD
GO

UPDATE Person.PersonPhone
SET PhoneNumber = '330-333-3333'

Commands completed successfully.

(19972 rows affected)

Total execution time: 00:00:02.612

## Uh-oh - my data!

In [4]:
USE AdventureWorks2017

SELECT TOP 10 *
FROM Person.PersonPhone

SELECT TOP 10 *
FROM Person.Password



(10 rows affected)

Total execution time: 00:00:00.025

BusinessEntityID,PhoneNumber,PhoneNumberTypeID,ModifiedDate
1,330-333-3333,1,2009-01-07 00:00:00.000
2,330-333-3333,3,2008-01-24 00:00:00.000
3,330-333-3333,1,2007-11-04 00:00:00.000
4,330-333-3333,1,2007-11-28 00:00:00.000
5,330-333-3333,1,2007-12-30 00:00:00.000
6,330-333-3333,3,2013-12-16 00:00:00.000
7,330-333-3333,3,2009-02-01 00:00:00.000
8,330-333-3333,1,2008-12-22 00:00:00.000
9,330-333-3333,1,2009-01-09 00:00:00.000
10,330-333-3333,3,2009-04-26 00:00:00.000


: Msg 208, Level 16, State 1, Line 6
Invalid object name 'Person.Password'.

## Let's Fix this from our snapshot
We have two options here
1. Pull certain data out of the snapshot to fix our mistakes
1. Revert the whole databases to the point in time the snapshot was taken

### Option 1 - Get the data to fix PersonPhone

In [ ]:
-- Clear out the data
DELETE 
FROM AdventureWorks2017.Person.PersonPhone

-- Insert the data from the table within our snapshot
INSERT INTO Person.PersonPhone
SELECT *
FROM AdventureWorks2017_20211007_141350.Person.PersonPhone

-- Check the data
SELECT TOP 10 * 
FROM AdventureWorks2017.Person.PersonPhone


### Option 2 - Revert to the time the snapshot was taken
Back to PowerShell...

## View the data again

In [ ]:
USE AdventureWorks2017

SELECT TOP 10 *
FROM Person.PersonPhone

SELECT TOP 10 *
FROM Person.Password